In [ ]:
import argparse
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import skimage
import yaml
from compartmentalizer.create_compartment import run_comaprtement_analysis
from matplotlib import pyplot as plt
from phenoscapes.cli import load_default_config, process_sample
from phenoscapes.feature_extraction import extract_features
from phenoscapes.sc import convert_to_h5ad, plot_summary
from phenoscapes.segmentation import generate_label_overlay, run_cellpose
from phenoscapes.utils import get_metadata
from skimage import io
from skimage.measure import regionprops
from tqdm import tqdm

In [ ]:
os.chdir("/Brain_ECM_4i_2_v2/")
dir_output = os.getcwd()

regex_exp = r"Stitched_C(?P<cycle_id>\d{1,2})_R(?P<well_id>\d{2,3})_ch(?P<channel_id>\d{2})\.tif$"
# load default config file
config = load_default_config()
# add regex to config
config["regex"] = regex_exp
# add path to stain metadata to config
config["stain_metadata"] = str(Path(dir_output, "stain_metadata.csv"))
# set nuclei channel
config["channel_nuclei"] = 0
# set staining channels
config["channel_stains"] = [1, 2, 3]
# set stain nuclei
config["stain_nuclei"] = "DAPI"
# Set ref cycle
config["reference_cycle"] = 0
# Set alignemnt ref cycles
config["alignment"]["reference_cycles"] = None
# Set parameter maps
config["alignment"]["param_maps"] = {
    "rigid": "/param_maps/translation.txt",
    "affine": "/param_maps/affine.txt",
}
# Cycle DAPI
config["simple_masking"]["sigma"] = 50
config["simple_masking"]["n_binary_operations"] = 50

config["refined_masking"]["cycle_dapi"] = 0
config["refined_masking"]["sigma"] = 50
config["refined_masking"]["n_binary_operations"] = 30
config["refined_masking"]["use_only_dapi"] = True
config["cropping_denoising"]["n_processes"] = 3

config["analysis_pipeline"]["slice_step"] = 1000
config["cropping_denoising"]["mask_zeros_smo"] = True
config["analysis_pipeline"]["stat"] = "mean"
config["speckle_removal"]["percentile"] = 3
config["feature_extraction"]["expand_labels"] = None

# Only run masking
config["stitching"]["run"] = True
config["simple_masking"]["run"] = True
config["alignment"]["run"] = True
config["refined_masking"]["run"] = True
config["alignment_check"]["run"] = True
config["cropping_denoising"]["run"] = True
config["speckle_removal"]["run"] = True
config["bg_subtraction"]["run"] = True
config["cellpose"]["run"] = True
config["feature_extraction"]["run"] = True
config["analysis_pipeline"]["run"] = True
config["analysis_pipeline"]["run"] = True
config["feature_morphology_extraction"]["run"] = True
default_config = config

df_stains = pd.read_csv("stain_metadata.csv")

In [ ]:
dir_bg_subtracted = Path(dir_output, "bg_subtracted")
samples = [  # Day 7
    "R066_0",
    "R062_0",
    "R073_0",
    "R072_0",
    # Day 15
    "R026_0",
    "R022_0",
    "R024_0",
    "R049_0",
    "R055_0",
    "R044_0",
    "R057_0",
    # Day 21
    "R076_1",
    "R087_0",
    "R082_0",
    "R108_1",
    "R112_1",
    "R113_1",
    "R115_0",
]

for sample in tqdm(samples):
    run_comaprtement_analysis(sample, default_config)